### Use Evaluation

In [15]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [39]:
#Set config. 
import sys, datetime, os, importlib
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 9999)
sys.path.append(os.path.join(Path.cwd().parent, 'modules'))
import evaluation as ev
importlib.reload(ev)
#These are configuration prediction files
c_p={'dir': '../data/predictions/tests/',
         'file': 'tests_75_shift0.csv',
         'per_col': 'person_id',
         'date_col': 'yyyymm',
         'date_for': '%Y%m'
        }
#These are configuration referral files
c_r={ 'dir': '../data/referrals/',
         'file': 'test.csv',
         'per_col': 'person_id',
         'date_col_in': 'date',
         'date_for_in': '%m/%d/%Y',
         'date_col_out': 'yyyymm', 
         'date_for_out': '%Y%m', 
         'reason_col': 'class'
        }
#These are configuration for the experiment
c_e={    'experiment': 'Initial Tests',
         'dir': '../results/',
         'file': 'results.csv',
         'save':  True,
         'append': False,
         'thresholds': [0.5, 0.6],
         'k': [5, 10], #k is used the set the conversion from prob to class prediction
         'landmarks': [[0,3], [0,6], [0,12]],
         'ref_target': 'ref',
         'pred_target': 'pref',
         'eval_date': datetime.date(2017, 1,1)
        }

In [40]:
#This is the truth 
ref=pd.read_csv(c_r['dir']+c_r['file'])
ref

,person_id,date,class
0,1004,1/1/2017,diabetes
1,1004,1/2/2017,liver
2,1004,6/2/2017,liver
3,1005,1/1/2017,liver
4,1005,1/2/2017,diabetes
5,1006,6/1/2017,liver
6,1007,6/1/2017,liver
7,1007,6/2/2017,liver
8,2000,10/1/2017,pnemonia
9,2001,10/1/2017,pnemonia


In [41]:
#These are the predictions. 
pred=pd.read_csv(c_p['dir']+c_p['file'])
pred['datetime']=pd.to_datetime(pred[c_p['date_col']], format= c_p['date_for'])
pred=pred.loc[pred['datetime']==c_e['eval_date'],[c_p['per_col'],'datetime',c_e['pred_target']]]
pred

,person_id,datetime,pref
12,1000,2017-01-01,0.00
36,1001,2017-01-01,0.00
60,1002,2017-01-01,0.00
84,1003,2017-01-01,0.00
108,1004,2017-01-01,0.75
132,1005,2017-01-01,0.75
156,1006,2017-01-01,0.00
180,1007,2017-01-01,0.00
204,1008,2017-01-01,0.00
228,1009,2017-01-01,0.00


In [42]:
importlib.reload(ev)
results = ev.score_times(c_p, c_r, c_e)
results

Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_75_shift0.csv
Evaluating from: 201701-201704
Evaluating from: 201701-201707
Evaluating from: 201701-201801


,experiment,date,pred_dir,pred_file,n,range,log_loss,roc_auc_score,precision_ref_@k=5,recall_ref_@k=5,accuracy_ref_@k=5,balanced_accuracy_ref_@k=5,f1_ref_@k=5,precision_ref_@k=10,recall_ref_@k=10,accuracy_ref_@k=10,balanced_accuracy_ref_@k=10,f1_ref_@k=10,precision_ref_p>0.5,recall_ref_p>0.5,accuracy_ref_p>0.5,balanced_accuracy_ref_p>0.5,f1_ref_p>0.5,precision_ref_p>0.6,recall_ref_p>0.6,accuracy_ref_p>0.6,balanced_accuracy_ref_p>0.6,f1_ref_p>0.6
0,Initial Tests,2020-03-10 21:40:02.744280,../data/predictions/tests/,tests_75_shift0.csv,12.0,201701-201704,0.047947,1.000000,0.4,1.000000,0.750000,0.850000,0.571429,0.2,1.0,0.333333,0.600000,0.333333,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
1,Initial Tests,2020-03-10 21:40:02.814223,../data/predictions/tests/,tests_75_shift0.csv,12.0,201701-201707,5.804410,0.750000,0.4,0.500000,0.583333,0.562500,0.444444,0.4,1.0,0.500000,0.625000,0.571429,1.0,0.500000,0.833333,0.750000,0.666667,1.0,0.500000,0.833333,0.750000,0.666667
2,Initial Tests,2020-03-10 21:40:02.864014,../data/predictions/tests/,tests_75_shift0.csv,12.0,201701-201801,11.560872,0.666667,0.4,0.333333,0.416667,0.416667,0.363636,0.6,1.0,0.666667,0.666667,0.750000,1.0,0.333333,0.666667,0.666667,0.500000,1.0,0.333333,0.666667,0.666667,0.500000


# Multiple different evaluations
This is an example of multiple different evaluations. This evaluates a bunch of files where the predictions have been shifted. 


In [43]:
importlib.reload(ev)
c_e['experiment']='All Tests'
c_e['append']= True
c_e['file']= 'all_results.csv'

pred_files = [f for f in os.listdir(c_p['dir']) if os.path.isfile(os.path.join(c_p['dir'], f))]

for file in pred_files:
    c_p['file']=file
    results = ev.score_times(c_p, c_r, c_e)

Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_75_shift-6.csv
Evaluating from: 201701-201704
Evaluating from: 201701-201707
Evaluating from: 201701-201801
Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_75_shift-12.csv
Evaluating from: 201701-201704
Evaluating from: 201701-201707
Evaluating from: 201701-201801
Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_75_shift0.csv
Evaluating from: 201701-201704
Evaluating from: 201701-201707
Evaluating from: 201701-201801
Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_75_shift6.csv
Evaluating from: 201701-201704
Evaluating from: 201701-201707
Evaluating from: 201701-201801
Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_100_shift-6.csv
Evaluating from: 201701-201704
Evaluating from: 201701-201707
Evaluating from: 201701-201801
Loading reference dataframe.. test.csv
Loading predictions dataframe.. test

In [44]:
#read in the full results
results=pd.read_csv(c_e['dir']+c_e['file'])
results

,experiment,date,pred_dir,pred_file,n,range,log_loss,roc_auc_score,precision_ref_@k=5,recall_ref_@k=5,accuracy_ref_@k=5,balanced_accuracy_ref_@k=5,f1_ref_@k=5,precision_ref_@k=10,recall_ref_@k=10,accuracy_ref_@k=10,balanced_accuracy_ref_@k=10,f1_ref_@k=10,precision_ref_p>0.5,recall_ref_p>0.5,accuracy_ref_p>0.5,balanced_accuracy_ref_p>0.5,f1_ref_p>0.5,precision_ref_p>0.6,recall_ref_p>0.6,accuracy_ref_p>0.6,balanced_accuracy_ref_p>0.6,f1_ref_p>0.6
0,All Tests,2020-03-10 21:40:44.378292,../data/predictions/tests/,tests_75_shift-6.csv,12.0,201701-201704,5.756463e+00,0.500000,0.2,0.500000,0.583333,0.550000,0.285714,0.2,1.0,0.333333,0.600000,0.333333,0.0,0.000000,0.833333,0.500000,0.000000,0.0,0.000000,0.833333,0.500000,0.000000
1,All Tests,2020-03-10 21:40:44.456772,../data/predictions/tests/,tests_75_shift-6.csv,12.0,201701-201707,1.151293e+01,0.500000,0.2,0.250000,0.416667,0.375000,0.222222,0.4,1.0,0.500000,0.625000,0.571429,0.0,0.000000,0.666667,0.500000,0.000000,0.0,0.000000,0.666667,0.500000,0.000000
2,All Tests,2020-03-10 21:40:44.498142,../data/predictions/tests/,tests_75_shift-6.csv,12.0,201701-201801,1.726939e+01,0.500000,0.4,0.333333,0.416667,0.416667,0.363636,0.6,1.0,0.666667,0.666667,0.750000,0.0,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000
3,All Tests,2020-03-10 21:40:44.594698,../data/predictions/tests/,tests_75_shift-12.csv,12.0,201701-201704,5.756463e+00,0.500000,0.2,0.500000,0.583333,0.550000,0.285714,0.2,1.0,0.333333,0.600000,0.333333,0.0,0.000000,0.833333,0.500000,0.000000,0.0,0.000000,0.833333,0.500000,0.000000
4,All Tests,2020-03-10 21:40:44.674020,../data/predictions/tests/,tests_75_shift-12.csv,12.0,201701-201707,1.151293e+01,0.500000,0.2,0.250000,0.416667,0.375000,0.222222,0.4,1.0,0.500000,0.625000,0.571429,0.0,0.000000,0.666667,0.500000,0.000000,0.0,0.000000,0.666667,0.500000,0.000000
5,All Tests,2020-03-10 21:40:44.722791,../data/predictions/tests/,tests_75_shift-12.csv,12.0,201701-201801,1.726939e+01,0.500000,0.4,0.333333,0.416667,0.416667,0.363636,0.6,1.0,0.666667,0.666667,0.750000,0.0,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000
6,All Tests,2020-03-10 21:40:44.829452,../data/predictions/tests/,tests_75_shift0.csv,12.0,201701-201704,4.794701e-02,1.000000,0.4,1.000000,0.750000,0.850000,0.571429,0.2,1.0,0.333333,0.600000,0.333333,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
7,All Tests,2020-03-10 21:40:44.898853,../data/predictions/tests/,tests_75_shift0.csv,12.0,201701-201707,5.804410e+00,0.750000,0.4,0.500000,0.583333,0.562500,0.444444,0.4,1.0,0.500000,0.625000,0.571429,1.0,0.500000,0.833333,0.750000,0.666667,1.0,0.500000,0.833333,0.750000,0.666667
8,All Tests,2020-03-10 21:40:44.940647,../data/predictions/tests/,tests_75_shift0.csv,12.0,201701-201801,1.156087e+01,0.666667,0.4,0.333333,0.416667,0.416667,0.363636,0.6,1.0,0.666667,0.666667,0.750000,1.0,0.333333,0.666667,0.666667,0.500000,1.0,0.333333,0.666667,0.666667,0.500000
9,All Tests,2020-03-10 21:40:45.034814,../data/predictions/tests/,tests_75_shift6.csv,12.0,201701-201704,5.756463e+00,0.500000,0.2,0.500000,0.583333,0.550000,0.285714,0.2,1.0,0.333333,0.600000,0.333333,0.0,0.000000,0.833333,0.500000,0.000000,0.0,0.000000,0.833333,0.500000,0.000000
